# Multi object tracking(MOT) analysis Sample Application

This notebook shows how to create an analysis app for Panorama using a pretrained PyTorch model.

--- 

1. [Prerequisites](#Prerequisites)
1. [Set up](#Set-up)
1. [Import model](#Import-model)
1. [Write and test app code](#Write-and-test-app-code-in-notebook)
1. [Package app](#Package-app)
1. [Deploy app to device](#Deploy-app-to-device)

# Prerequisites

1. In a terminal session on this Jupyter notebook server, run `aws configure`. This allows this notebook server to access Panorama resources and deploy applications on your behalf.

# Set up

Import libraries for use with this notebook environment, you do not need these libraries when you write your application code.

In [ ]:
import sys
import os
import time
import json

import boto3
import sagemaker

import matplotlib.pyplot as plt
from IPython.core.magic import register_cell_magic

# configure matplotlib
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,20)

# register custom magic command
@register_cell_magic
def save_cell(line, cell):
    'Save python code block to a file'
    with open(line, 'wt') as fd:
        fd.write(cell)

## Notebook parameters
Global constants that help the notebook create Panorama resources on your behalf.

In [ ]:
# Device ID, should look like: device-oc66nax4cgzwhyuaeyifrqowue
DEVICE_ID = input( 'DEVICE_ID (format: device-*)' ).strip()

# Enter your S3 bucket info here
S3_BUCKET = input( 'S3_BUCKET' ).strip()

# Enter your desired AWS Panorama region
AWS_REGION = input( 'AWS_REGION (e.g. us-east-1)' ).strip()

# Enter application role to be deployed in panorama device (S3 and kinesis firehose required, secretsmanager and kinesis video are optional if want remote view)
APPLICATION_ROLE = input( 'Application ROLE ARN' ).strip()

# Precompiled sagemaker neo compatible yolox models are yolox_s_neo and yolox_m_neo
ML_MODEL_FNAME = 'yolox_m_neo' 

In [ ]:
model_package_name = 'YOLOXM_MODEL'
model_data_shape = '{"input0":[1,3,640,640]}'

# application name
app_name = 'mot_analysis_app'

## package names and node names
code_package_name = 'MOT_ANALYSIS_CODE'
camera_node_name = 'abstract_rtsp_media_source'

container_asset_name = 'code_asset'

# model node name, raw model path (without platform dependent suffics), and input data shape
model_node_name = "model_node"
model_file_basename = "./models/" + ML_MODEL_FNAME

# video file path to simulate camera stream
videoname = '../common/test_utility/videos/TownCentreXVID.avi'

sys.path.insert( 0, os.path.abspath( "../common/test_utility" ) )
import panorama_test_utility

# instantiate boto3 clients
s3_client = boto3.client('s3')
panorama_client = boto3.client('panorama', region_name=AWS_REGION)

# AWS account ID
account_id = boto3.client("sts").get_caller_identity()["Account"]

## Set up application

Every application uses the creator's AWS Account ID as the prefix to uniquely identifies the application resources. Running `panorama-cli import-application` replaces the generic account Id with your account Id.

In [ ]:
!cd ./{app_name} && panorama-cli import-application

# Import model

We need to compile and import the model twice. Once for testing with this notebook server and once for deploying to the Panorama device.

While working with the Panorama sample code, we provide pretrained models for you to use. Locally, models are stored in `./models`. This step downloads the model artifacts from our Amazon S3 bucket to the local folder. If you want to use your own models, put your tar.gz file into the `./models` folder.

### Prepare model for testing with notebook server

In [ ]:
# Downloads pretrained model for this sample.
# This step takes some time, depending on your network environment.
panorama_test_utility.download_sample_model( ML_MODEL_FNAME, "./models" )

In [ ]:
# Compile the model to run with test-utility.
# This step takes 7 mins ~ 10 mins.
%run ../common/test_utility/panorama_test_utility_compile.py \
\
--s3-model-location s3://{S3_BUCKET}/{app_name}/ \
\
--model-node-name model_node \
--model-file-basename ./models/{ML_MODEL_FNAME} \
--model-data-shape '{model_data_shape}' \
--model-framework pytorch

### Prepare model for deploying to Panorama device

In [ ]:
model_asset_name = 'model_asset'
model_package_path = f'packages/{account_id}-{model_package_name}-1.0'
model_descriptor_path = f'packages/{account_id}-{model_package_name}-1.0/descriptor.json'

In [ ]:
!cd ./{app_name} && panorama-cli add-raw-model \
    --model-asset-name {model_asset_name} \
    --model-local-path ../models/{ML_MODEL_FNAME}.tar.gz \
    --descriptor-path {model_descriptor_path}  \
    --packages-path {model_package_path}

# Write and test app code in notebook

Every app has an entry point script, written in Python that pulls the frames from camera streams, performs inference, and send the results to the desired location. This file can be found in `your_app/packages/code_node/src/app.py`. Below, you will iterate on the code from within the notebook environment. The entry point file will be updated everytime you run the next notebook cell thanks to the `%%save_cell`. This is a  magic command to update the contents of the entry point script. 

After updating the entry point script, use the Test Utility Run command (panorama_test_utility_run.py) command to simulate the application.

### Iterating on Code Changes

To iterate on the code:
1. Interrupt the kernel if application is still running.
2. Make changes in the next cell, and run the cell to update the entry point script. 
3. Run the panorama_test_utility_run.py again.

**CHANGE VIDEO** : For you to change video, please set the file path to the --video-file argument of the panorama_test_utility_run.py command.

In [ ]:
%%save_cell ./{app_name}/packages/{account_id}-{code_package_name}-1.0/src/app.py

import json
import logging
import time
from logging.handlers import RotatingFileHandler

import boto3
import cv2
import numpy as np
import panoramasdk

import os
os.environ["GST_DEBUG"] = "2"
os.environ["GST_PLUGIN_PATH"] = "$GST_PLUGIN_PATH:/usr/local/lib/gstreamer-1.0/:/amazon-kinesis-video-streams-producer-sdk-cpp/build"

from types import SimpleNamespace
import torch
from bytetracker.byte_tracker import BYTETracker
from yolox_postprocess import demo_postprocess, multiclass_nms

from datetime import datetime, timezone

class Application(panoramasdk.node):
    def __init__(self):
        """Initializes the application's attributes with parameters from the interface, and default values."""
        self.VIDEO_RECORDING = False
        self.STREAM_ID = 0
        self.MODEL_NODE = "model_node"
        self.MODEL_INPUT = (640, 640)  #YOLOX
        self.source_fnum = 0
        self.target_fnum = 0
        
        #for uploading still-shot each start day
        self.refresh = True
        self.lastday = datetime.now(timezone.utc).strftime('%Y-%m-%d')
        self.today = self.lastday
        
        #Origin size
        streams = self.inputs.video_in.get()
        stream = streams[0]
        width = stream.image.shape[1]
        height = stream.image.shape[0]
        
        self.CAMERA_INPUT = (height, width)
        
        # Parameters
        logger.info('Getting parameters')
        self.service_region = self.inputs.service_region.get()
        self.bucket_name = self.inputs.bucket_name.get()
        self.kinesis_name = self.inputs.kinesis_name.get()
        self.kinesis_video_name = self.inputs.kinesis_video_name.get()
        
        session = boto3.Session(region_name=self.service_region)
        self.s3 = session.client('s3')
        self.firehose = session.client('firehose')
            
        self.SOURCE_FPS = self.inputs.source_fps.get() #30
        self.TARGET_FPS = self.inputs.target_fps.get() #10
        self.CATEGORY = compile(self.inputs.yolox_category.get(), '<string>', 'eval') #[0,1,2]
        self.VERTICAL_RATIO = round(self.inputs.vertical_ratio.get(), 2) #1.6
        
        self.args = SimpleNamespace(**{
                    'nms': round(self.inputs.nms.get(),2), #0.45
                    'track_thresh': round(self.inputs.track_thresh.get(), 2), #0.65
                    'track_buffer': self.inputs.track_buffer.get(), #30
                    'match_thresh': round(self.inputs.match_thresh.get(), 2), #0.9
                    'min_box_area': self.inputs.min_box_area.get(), #100 w*h
                    'mot20': False})
            
        self.trackers = [BYTETracker(self.args, frame_rate=self.TARGET_FPS) for _ in range(len(streams))]
        
        gst_out = self.inputs.gstreamer_encoder.get()
        if len(gst_out) > 0:
            self.VIDEO_RECORDING = True
            kvssecret = session.client('secretsmanager')                
            aksk = json.loads(kvssecret.get_secret_value(SecretId='KVSSecret')['SecretString'])
            gst_out += f" ! kvssink log-config=/amazon-kinesis-video-streams-producer-sdk-cpp stream-name={self.kinesis_video_name} framerate={self.TARGET_FPS} access-key={aksk['accesskey']} secret-key={aksk['secretkey']} aws-region={self.service_region} "
            self.videowriter = cv2.VideoWriter(gst_out, cv2.CAP_GSTREAMER, 0, float(self.TARGET_FPS), (width, height))
        
        logger.info('Initialiation complete.')
        logger.info('Args: {}'.format(self.args))

    def stop(self):
        if self.VIDEO_RECORDING == True:
            self.videowriter.release()
        logger.info('Terminated.')
        
    def resetstate(self):
        streams = self.inputs.video_in.get()
        for stream, tracker in zip(streams, self.trackers):
            image = cv2.imencode('.png', stream.image)[1].tostring()
            self.s3.put_object(Body=image, Bucket=self.bucket_name, 
                          Key=f"dailycapture/{stream.stream_id}/{self.today}.png", ContentType='image/PNG')
            #Refresh byte_track
            tracker.reset()

    def process_streams(self):
        """Processes one frame of video from one or more video streams."""
        self.source_fnum += 1
        
        #Check every minute to refresh check, 30 * 60 is 60 seconds
        if self.source_fnum % 1800 == 0:
            self.today = datetime.now(timezone.utc).strftime('%Y-%m-%d')
            if self.lastday != self.today:
                self.lastday = self.today
                self.refresh = True
        
        if self.refresh == True:
            self.refresh = False
            self.resetstate()
            
        #For processing partial frames to improve performance
        if self.source_fnum % (self.SOURCE_FPS / self.TARGET_FPS) != 0:
            return
        
        self.target_fnum += 1

        # Loop through attached video streams
        streams = self.inputs.video_in.get()
        for stream, tracker in zip(streams, self.trackers):
            self.process_media(stream, tracker)

        #TODO: Currently send only stream 0 to KVS, additional implemendation required to switch stream id by using iot channel
        if self.VIDEO_RECORDING == True:
            self.videowriter.write(streams[self.STREAM_ID].image)
        
        self.outputs.video_out.put(streams)
    
    def preproc(self, img, input_size, swap=(2, 0, 1)):
        if len(img.shape) == 3:
            padded_img = np.ones((input_size[0], input_size[1], 3)) * 114.0
        else:
            padded_img = np.ones(input_size) * 114.0
        
        r = min(input_size[0] / img.shape[0], input_size[1] / img.shape[1])
        resized_img = cv2.resize(
            img,
            (int(img.shape[1] * r), int(img.shape[0] * r)),
            interpolation=cv2.INTER_LINEAR,
        ).astype(np.uint8)
        padded_img[: int(img.shape[0] * r), : int(img.shape[1] * r)] = resized_img

        padded_img = padded_img.transpose(swap)
        padded_img = np.ascontiguousarray(padded_img, dtype=np.float32)
        return padded_img, r
    
    def process_media(self, stream, tracker):
        """Runs inference on a frame of video."""
        image_data, ratio = self.preproc(stream.image, self.MODEL_INPUT)

        inference_results = self.call({"input0":image_data}, self.MODEL_NODE)[0]
        
        # Process results (object deteciton)
        num_people = 0
        if len(inference_results) > 0:
            num_people = self.process_results(inference_results, stream, tracker, ratio)            
        
        add_label(stream.image, f"{stream.stream_id} / # People {num_people} / {datetime.utcnow().strftime('%H:%M:%S.%f')[:-5]}", 30, 50)
    
    def process_results(self, inference_results, stream, tracker, ratio):
        boxes, scores, class_indices = self.postprocess(inference_results, self.MODEL_INPUT, ratio)        
        if boxes is None:
            return 0
        
        media_height, media_width, _ = stream.image.shape
        media_scale = np.asarray([media_width, media_height, media_width, media_height])
        
        candidates = []
        for box, score, category_id in zip(boxes, scores, class_indices):
            if category_id in eval(self.CATEGORY):
                w = box[2] - box[0]
                h = box[3] - box[1]
                if w * h < self.args.min_box_area:
                    continue
                horizontal = w / h > self.VERTICAL_RATIO
                if category_id == 0 and horizontal:
                    continue
                candidates.append([box[0], box[1], box[2], box[3], score, category_id])
        
        num_people = len(candidates)
        if num_people == 0:
            return 0
        
        online_targets = tracker.update(self.target_fnum, torch.tensor(candidates))
        jsonlist = []
        ts = stream.time_stamp
        for t in online_targets:
            tlwh = t.tlwh
            tid = t.track_id
            tcid = t.category_id
            tscore = t.score
            age = round((self.target_fnum - t.start_frame)/self.TARGET_FPS, 1)
            jsonlist.append({"Data":f'{{"sid":"{stream.stream_id}","ts":{ts[0] + (0.1) * ts[1]},"fnum":{self.target_fnum},"cid":{tcid},"tid":{tid},"age":{age},"left":{tlwh[0]/self.CAMERA_INPUT[1]},"top":{tlwh[1]/self.CAMERA_INPUT[0]},"w":{tlwh[2]/self.CAMERA_INPUT[1]},"h":{tlwh[3]/self.CAMERA_INPUT[0]}}}'})
            add_rect(stream.image, tlwh[0], tlwh[1], tlwh[2], tlwh[3])
            add_label(stream.image, f'{tid}/{tcid}/{age}', tlwh[0], tlwh[1] - 10)

        num_people = len(jsonlist)
        if num_people == 0:
            return 0
        
        self.firehose.put_record_batch(DeliveryStreamName=self.kinesis_name, Records=jsonlist)
        
        return num_people
        
    def postprocess(self, result, input_shape, ratio):        
        # source: https://github.com/Megvii-BaseDetection/YOLOX/blob/2c2dd1397ab090b553c6e6ecfca8184fe83800e1/demo/ONNXRuntime/onnx_inference.py#L73
        input_size = input_shape[-2:]
        predictions = demo_postprocess(result, input_size)
        predictions = predictions[0] # TODO: iterate through eventual batches
                
        boxes = predictions[:, :4]
        scores = predictions[:, 4:5] * predictions[:, 5:]

        boxes_xyxy = np.ones_like(boxes)
        boxes_xyxy[:, 0] = boxes[:, 0] - boxes[:, 2]/2.
        boxes_xyxy[:, 1] = boxes[:, 1] - boxes[:, 3]/2.
        boxes_xyxy[:, 2] = boxes[:, 0] + boxes[:, 2]/2.
        boxes_xyxy[:, 3] = boxes[:, 1] + boxes[:, 3]/2.
        boxes_xyxy /= ratio
        
        dets = multiclass_nms(boxes_xyxy, scores, nms_thr=self.args.nms, score_thr=0.1)
        if dets is None:
            return None, None, None
        
        final_boxes, final_scores, final_cls_inds = dets[:, :4], dets[:, 4], dets[:, 5]
        boxes = final_boxes
        scores = final_scores
        class_indices = final_cls_inds.astype(int)
        return boxes, scores, class_indices
    
def add_label(image, text, x1, y1):
    # White in BGR
    color = (255, 255, 255)
    # Using cv2.putText() method
    return cv2.putText(image, text, (int(x1), int(y1)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

def add_rect(image, x1, y1, x2, y2):
    # Red in BGR
    color = (0, 0, 255)        
    return cv2.rectangle(image, (int(x1), int(y1)), (int(x1 + x2), int(y1 + y2)), color, 2)

def get_logger(name=__name__,level=logging.INFO):
    logger = logging.getLogger(name)
    logger.setLevel(level)
    handler = RotatingFileHandler("/opt/aws/panorama/logs/app.log", maxBytes=100000000, backupCount=2)
    formatter = logging.Formatter(fmt='%(asctime)s %(levelname)-8s %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    return logger

def main():
    while True:
        try:
            logger.info("INITIALIZING APPLICATION")
            app = Application()
            logger.info("PROCESSING STREAMS")
            while True:
                app.process_streams()
        except:
            logger.exception('Exception during processing loop.')
        finally:
            app.stop()
        
        #TODO: What about the failover?
        break
        #time.sleep(10)

logger = get_logger(level=logging.INFO)
main()

In [ ]:
# Run the application with test-utility.
#
# As '--output-pyplot' option is specified, this command simulates HDMI output with pyplot rendering in the output cell.
# In order to see console output (stdout/stderr) from the application, please remove the --output-pyplot option.
#
%run ../common/test_utility/panorama_test_utility_run.py \
\
--app-name {app_name} \
--code-package-name {code_package_name} \
--model-package-name {model_package_name} \
--camera-node-name {camera_node_name} \
--model-node-name {model_node_name} \
--model-file-basename {model_file_basename} \
--video-file {videoname} \
--video-stop 30 \
--py-file ./{app_name}/packages/{account_id}-{code_package_name}-1.0/src/app.py \
--output-pyplot

# Package app

Updates the app to be deployed with the recent code

In [ ]:
py_file_name = 'app.py'
panorama_test_utility.update_package_descriptor( app_name, account_id, code_package_name, py_file_name )

## Update camera streams

In the AWS Panorama console, you can select the camera streams, but programmatically, you need to define the camera stream info for the cameras you are using with the app.

We used an ```abstract data source``` here, usually this lets you select the pre-created camera source from the console. But programatically, we have to do the following steps


- Create Camera
- Create Override json file
- Include the Override json file while are deploying the application

### Create New Camera

Because we are using an ```abstract_rtsp_media_source```, we have to create a camera before we can use the ```abstract_rtsp_media_source```

**NOTE** : Update your RTSP Info in the next cell, Username, Password and RTSP Stream URL

In [ ]:
#For testing purpose, use https://github.com/aler9/rtsp-simple-server to generate infitite live video using static file
CAMERA_NAME = "StreetSample"
CAMERA_CREDS = '{"StreamUrl": "rtsp://192.168.35.44:8554/mystream"}'

In [ ]:
res = !aws panorama create-node-from-template-job --template-type RTSP_CAMERA_STREAM \
    --output-package-name {CAMERA_NAME} \
    --output-package-version '1.0' \
    --node-name {CAMERA_NAME} \
    --template-parameters '{CAMERA_CREDS}'

# FIXME : camera node creation fails if it already exists.
# Should either ignore the already-exist error, or delete the node at the end of this notebook

res = ''.join(res)
print(res)
res_json = json.loads(res)

In [ ]:
!aws panorama describe-node-from-template-job --job-id {res_json['JobId']}

## Overriding camera node

If you want to override the camera configuration at deployment (for ex. deploy to another site) you can provide a deployment time override. Go to `mot_analysis_app/deployment_overrides/override_camera.json` file and replace YOUR_AWS_ACCOUNT_ID with your ACCOUNT_ID and YOUR_CAMERA_NAME with your camera name.

In [ ]:
# Update Account ID
with open( f"./{app_name}/deployment_overrides/override_camera.json", "r" ) as fd:
    override_json = json.load(fd)

override_json['nodeGraphOverrides']['packages'][0]['name'] = '{}::{}'.format(account_id, CAMERA_NAME)
override_json['nodeGraphOverrides']['nodes'][0]['name'] = CAMERA_NAME
override_json['nodeGraphOverrides']['nodes'][0]['interface'] = '{}::{}.{}'.format(account_id, CAMERA_NAME, CAMERA_NAME)    
override_json['nodeGraphOverrides']['nodeOverrides'][0]['with'][0]['name'] = CAMERA_NAME    

with open( f"./{app_name}/deployment_overrides/override_camera.json", "w") as fd:
    json.dump(override_json, fd)

### Build app with container

In [ ]:
%%capture captured_output

# Building container image.This process takes time (5min ~ 10min)
# FIXME : without %%capture, browser tab crashes because of too much output from the command.

!cd ./{app_name} && panorama-cli build \
    --container-asset-name {container_asset_name} \
    --package-path packages/{account_id}-{code_package_name}-1.0

### Upload application to Panorama for deploying to devices

In [ ]:
# This step takes some time, depending on your network environment.
!cd ./{app_name} && panorama-cli package-application

### Ready for deploying to a device

Congrats! Your app is now ready to deploy to a device. Next, you can continue in this notebook to deploy the app programmatically or you can go to the Panorama console and deploying using the AWS Console. The console makes it easier to select camera streams and select the devices you want to deploy to. Programmatic deployment is faster to complete and easier to automate.

# Deploy app to device

Let's make sure the device we are deploying to is available.

In [ ]:
response = panorama_client.describe_device(
    DeviceId= DEVICE_ID
)

print('You are deploying to Device: {}'.format(response['Name']))

## Deploy app

You are ready to deploy your app. Below, you can see an example of how to use the AWS CLI to deploy the app. Alternatively, you can use the boto3 SDK as you did above for getting the device information.

In [ ]:
with open(f"./{app_name}/graphs/{app_name}/graph.json") as fd:
    manifest_payload = "'%s'" % json.dumps({"PayloadData":json.dumps(json.load(fd))})
    
with open(f"./{app_name}/deployment_overrides/override_camera.json") as fd:
    override_payload = "'%s'" % json.dumps({"PayloadData":json.dumps(json.load(fd))})

In [ ]:
res = !aws panorama create-application-instance \
    --name {app_name} \
    --default-runtime-context-device {DEVICE_ID} \
    --manifest-payload {manifest_payload} \
    --manifest-overrides-payload {override_payload} \
    --runtime-role-arn {APPLICATION_ROLE}

res = ''.join(res)
print(res)
res_json = json.loads(res)

### Check Application Status

In [ ]:
app_id = res_json['ApplicationInstanceId']
print( "Application Instance Id :", app_id )

progress_dots = panorama_test_utility.ProgressDots()
while True:
    response = panorama_client.describe_application_instance( ApplicationInstanceId = app_id )
    status = response['Status']
    progress_dots.update_status( f'{status} ({response["StatusDescription"]})' )
    if status in ('DEPLOYMENT_SUCCEEDED','DEPLOYMENT_FAILED'):
        break
    time.sleep(60)

# Clean up

In [ ]:
panorama_test_utility.remove_application( DEVICE_ID, app_id )

In [ ]:
nodes = panorama_client.list_nodes()
for node in nodes['Nodes']:
    print(f'Deleting {node["PackageId"]}')
    panorama_client.delete_package(ForceDelete=True, PackageId=node['PackageId'])